In [1]:
import sys
import os
#Add parent directory to path to enable imports
# Since we're in notebooks/ directory, go up one level
current_dir = os.getcwd()
# If we're in notebooks/, go up to parent
if current_dir.endswith('notebooks'):
    parent_dir = os.path.dirname(current_dir)
else:
    # If we're already in the root, use current directory
    parent_dir = current_dir

sys.path.insert(0, parent_dir)

In [2]:
from S_TickNet_Dogs import *
from util import get_device

In [3]:
EPOCH_SIZE = 1
BATCH_SIZE = 64
GPU_ID = 0
BASE_DIR = '../results'
DATA_ROOT = '../datasets/Cifar100'
CONFIG = 2
NETWORK = 'basic'
WORKERS = 4
LEARNING_RATE = 0.1
MOMENTUM = 0.9
WEIGHT_DECAY = 0.0001
SCHEDULE = [100, 150]
GAMMA = 0.1
DOWNLOAD = True
DATASET = 'cifar100'

In [4]:
device = get_device()
print(device)

mps


In [5]:
model = build_STickNet(120, typesize=NETWORK, cifar=True)
model = model.to(device)
model_params = sum([p.data.nelement() for p in model.parameters()])
print(model)
print('Number of model parameters: {}'.format(model_params))

THE ACTUAL CHANNEL: basic
THE config: [[256], [128], [64], [128, 256], [512]]
SpatialTickNet(
  (backbone): Sequential(
    (data_bn): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (init_conv): ConvBlock(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU(inplace=True)
    )
    (stage1): Sequential(
      (unit1): FR_PDP_block(
        (Pw1): ConvBlock(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        )
        (Dw): ConvBlock(
          (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activation): ReLU(inplace=True)
        )
        (Pw2): ConvBlock(
          (conv): Conv2d(32, 256, kernel_size=(1, 1)

In [6]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(
        params=model.parameters(),
        lr=LEARNING_RATE,
        momentum=MOMENTUM,
        weight_decay=WEIGHT_DECAY,
)

scheduler = torch.optim.lr_scheduler.MultiStepLR(
  optimizer=optimizer, milestones=SCHEDULE, gamma=0.1
)

# get train and val da  ta loaders
train_loader = get_data_loader(
    dataset_name=DATASET,
    data_root=DATA_ROOT,
    batch_size=BATCH_SIZE,
    workers=WORKERS,
    download=DOWNLOAD,
    train=True
)
val_loader = get_data_loader(
    dataset_name=DATASET,
    data_root=DATA_ROOT,
    batch_size=BATCH_SIZE,
    workers=WORKERS,
    download=DOWNLOAD,
    train=False
)

In [7]:
strmode = f'StanfordDogs_S_TickNet_{NETWORK}_SE'
pathout = f'{BASE_DIR}/checkpoints/{strmode}'
# Create directories if they don't exist
os.makedirs(pathout, exist_ok=True)

filenameLOG = pathout + '/' + strmode + '.txt'
result_file_path = pathout + '/' + strmode + '.csv'
val_accuracy_max = None
val_accuracy_argmax = None
for n_epoch in range(EPOCH_SIZE):
    current_learning_rate = optimizer.param_groups[0]['lr']
    print(
        f'Starting epoch {n_epoch + 1}/{EPOCH_SIZE}, learning_rate={current_learning_rate}'
    )
    # train
    (train_loss, train_accuracy) = run_epoch(
        train=True,
        data_loader=train_loader,
        model=model,
        criterion=criterion,
        optimizer=optimizer,
        n_epoch=n_epoch,
        total_epochs=EPOCH_SIZE,
        device=device,
    )

    # validate
    (val_loss, val_accuracy) = run_epoch(
        train=False,
        data_loader=val_loader,
        model=model,
        criterion=criterion,
        optimizer=None,
        n_epoch=n_epoch,
        total_epochs=EPOCH_SIZE,
        device=device,
    )
    if (val_accuracy_max is None) or (val_accuracy > val_accuracy_max):
      val_accuracy_max = val_accuracy
      val_accuracy_argmax = n_epoch
      torch.save(
          {"model_state_dict": model.state_dict()},
          f'{pathout}/checkpoint_epoch{n_epoch + 1:>04d}_{100.0 * val_accuracy_max:.2f}.pth',
      )

    # adjust learning rate
    scheduler.step()

    # print epoch summary
    line = (
        '=================================================================================='
        f'Epoch {n_epoch + 1}/{EPOCH_SIZE} summary: '
        f'loss_train={train_loss:.5f}, '
        f'acc_train={100.0 * train_accuracy:.2f}%, '
        f'loss_val={val_loss:.2f}, '
        f'acc_val={100.0 * val_accuracy:.2f}% '
        f'(best: {100.0 * val_accuracy_max:.2f}% @ epoch {(val_accuracy_argmax or 0) + 1})'
        '=================================================================================='
    )
    print(line)
    wA.writeLogAcc(filenameLOG, line)
    wA.log_results_to_csv(
        result_file_path,
        n_epoch + 1,
        train_loss,
        100.0 * train_accuracy,
        val_loss,
        100.0 * val_accuracy,
    )


Starting epoch 1/1, learning_rate=0.1
[train]  epoch 1/1,  batch 1/782,  loss_train=4.94707,  acc_train=1.56%
[train]  epoch 1/1,  batch 11/782,  loss_train=4.82665,  acc_train=4.69%
[train]  epoch 1/1,  batch 21/782,  loss_train=4.96397,  acc_train=3.12%
[train]  epoch 1/1,  batch 31/782,  loss_train=5.07506,  acc_train=0.00%
[train]  epoch 1/1,  batch 41/782,  loss_train=4.82221,  acc_train=0.00%
[train]  epoch 1/1,  batch 51/782,  loss_train=4.43169,  acc_train=7.81%
[train]  epoch 1/1,  batch 61/782,  loss_train=4.79113,  acc_train=1.56%
[train]  epoch 1/1,  batch 71/782,  loss_train=4.44360,  acc_train=3.12%
[train]  epoch 1/1,  batch 81/782,  loss_train=4.19709,  acc_train=6.25%
[train]  epoch 1/1,  batch 91/782,  loss_train=4.56934,  acc_train=3.12%
[train]  epoch 1/1,  batch 101/782,  loss_train=4.36267,  acc_train=6.25%
[train]  epoch 1/1,  batch 111/782,  loss_train=4.12534,  acc_train=6.25%
[train]  epoch 1/1,  batch 121/782,  loss_train=4.12657,  acc_train=9.38%
[train]  ep